In [47]:
import re

def parse_coq_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Extract components in order
    components = extract_components(content)
    return components

def extract_components(content):
    components = []
    patterns = [
            ('Comment', r'\(\*([\s\S]*?)\*\)'),
            ('Require', r'(Require\s+(?:Import|Export)?\s+[\w\.]+)\.'),
            ('Definition', r'(Definition\s+\w+(?:\s+[^:]+)?:\s*(?:[^\.]+))\.'),
            ('Theorem', r'(Theorem\s+\w+\s*:[\s\S]+?Proof\.)([\s\S]+?)(Qed\.)'),
            ('Lemma', r'(Lemma\s+\w+\s*:[\s\S]+?Proof\.)([\s\S]+?)(Qed\.)'),
            ('Inductive', r'(Inductive\s+\w+(?:\s+[^:]+)?:\s*(?:[^\.]+))\.'),
            ('Fixpoint', r'(Fixpoint\s+\w+(?:\s+[^:]+)?:\s*(?:[^\.]+))\.'),
            ('Notation', r'(Notation\s+[^\.]+)\.'),
            ('Hint', r'(Hint\s+[^\.]+)\.')
    ]

    while content:
        matched = False
        for component_type, pattern in patterns:
            match = re.match(pattern, content, re.DOTALL)
            if match:
                if component_type in ['Theorem', 'Lemma']:
                    components.append((component_type, parse_theorem_or_lemma(match)))
                else:
                    components.append((component_type, match.group(1).strip()))
                content = content[match.end():].lstrip()
                matched = True
                break
        if not matched:
            # Move past the current line if no match is found
            next_newline = content.find('\n')
            if next_newline == -1:
                break
            content = content[next_newline + 1:]

    return components

def parse_theorem_or_lemma(match):
    statement, proof, qed = match.groups()
    return {
            'Statement': statement.strip(),
            'Proof': {
                    'Steps': extract_proof_steps(proof),
                    'Qed': qed.strip()
            }
    }

def extract_proof_steps(proof):
    steps = re.split(r'\.(?:\s+|$)', proof)
    return [step.strip() for step in steps if step.strip()]

def print_results(components):
    for component_type, content in components:
        print(f"\n{component_type}:")
        if component_type in ['Theorem', 'Lemma']:
            print(f"  {content['Statement']}")
            print("  Proof:")
            for step in content['Proof']['Steps']:
                print(f"    {step}")
            print(f"  {content['Proof']['Qed']}")
        else:
            print(f"  {content}")


In [49]:
file_path = "dataset/community_repo/coq-community-exact-real-arithmetic-43bf40b/definition.v"
results = parse_coq_file(file_path)
print_results(results)


Comment:
  This code is copyrighted by its authors; it is distributed under

Comment:
  the terms of the LGPL license (see LICENSE and description files)

Require:
  Require Export ZArith

Require:
  Require Export Zcomplements

Require:
  Require Export Reals

Require:
  Require Export Omega

Require:
  Require Export Arith

Require:
  Require Export ZArith

Require:
  Require Export Zmisc

Require:
  Require Export Zpower

Require:
  Require Export Zmax

Definition:
  Definition nearest_Int (x : R) : Z := up (x - / 2)

Definition:
  Definition gauss_z_sur_B (x : Z) := nearest_Int (IZR x / INR B)

Definition:
  Definition B_powerRZ (z : Z) : R := powerRZ (INR B) z

Definition:
  Definition gauss_z_sur_B_pow x n := nearest_Int (IZR x / B_powerRZ n)

Lemma:
  Lemma gauss_correct :
   forall z : Z,
    (IZR z * / INR B - 1 * / 2 < IZR (gauss_z_sur_B z) <=
     IZR z * / INR B + 1 * / 2)%R.
Proof.
  Proof:
    intros z
    rewrite !Rmult_1_l
    now apply nearest_Int_correct
  Qed.

Lemm